# Glidelytics for Strava - paragliding analysis on personal Strava data
## Authorization 

Get client_id from
https://www.strava.com/settings/api

Visit page 
http://www.strava.com/oauth/authorize?client_id=[REPLACE_WITH_YOUR_CLIENT_ID]&response_type=code&redirect_uri=http://localhost/exchange_token&approval_prompt=force&scope=read_all
replacing placeholder with client_id

Get field "code"after redirection

Open file config.ini (or else create such file on root using example_config.ini). Update the file with the below fields and save:
- client_id
- client_secret
- code (from authorization response)

In [71]:
# Lets load the config info and refresh need

import configparser
config = configparser.ConfigParser()
config.read("config.ini")
print("Client ID: {}".format(config["strava"]["client_id"]))

from datetime import datetime

print(datetime.strptime(config["strava"]["expires_at"], "%H:%M:%S"))

Client ID: 80263


ValueError: time data '1648992339' does not match format '%H:%M:%S'

In [64]:
import requests
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

auth_url = "https://www.strava.com/oauth/token"
activites_url = "https://www.strava.com/api/v3/athlete/activities"


payload = {
    'client_id': config["strava"]["client_id"],
    'client_secret': config["strava"]["client_secret"],
    'refresh_token': config["strava"]["refresh_token"],
    'grant_type': "refresh_token",
    'f': 'json'
}

print("Requesting Token...\n")
res = requests.post(auth_url, data=payload, verify=False).json()
print(res)

Client ID: 80263
Requesting Token...

{'token_type': 'Bearer', 'access_token': '2fc52a5441f5671ccba8633ee45cf0674cdcdfb4', 'expires_at': 1648992339, 'expires_in': 8841, 'refresh_token': 'ee974d57080302b03e63eb0fac5981712fa7f7c0'}


In [68]:
print("Expires in: {} hours".format(str(resexpires_in / 3600)))



ImportError: cannot import name 'strftime' from 'datetime' (/usr/local/Cellar/python@3.9/3.9.12/Frameworks/Python.framework/Versions/3.9/lib/python3.9/datetime.py)

In [61]:
import requests
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

auth_url = "https://www.strava.com/oauth/token"
activites_url = "https://www.strava.com/api/v3/athlete/activities"



Requesting Token...

{'token_type': 'Bearer', 'access_token': '2fc52a5441f5671ccba8633ee45cf0674cdcdfb4', 'expires_at': 1648992339, 'expires_in': 9086, 'refresh_token': 'ee974d57080302b03e63eb0fac5981712fa7f7c0'}


In [62]:
access_token = res['access_token']
print("Access Token = {}\n".format(access_token))

header = {'Authorization': 'Bearer ' + access_token}
param = {'per_page': 200, 'page': 1}
my_dataset = requests.get(activites_url, headers=header, params=param).json()

Access Token = 2fc52a5441f5671ccba8633ee45cf0674cdcdfb4



In [63]:
print(my_dataset.head())

AttributeError: 'dict' object has no attribute 'head'

## Getting list of paragliding activities

In [49]:
# estimate of number of activities
act_total = 400
# activities per page
act_pp = 100
# number of pages
import math
pgs = math.ceil(act_total / act_pp)

# columns to keep from activities
cols = ["id", "name", "distance", "moving_time", "elapsed_time", "total_elevation_gain", "type", "start_date_local", "map.id", "average_speed", "max_speed", "elev_high", "elev_low", ""]

#request header
header = {'Authorization': 'Bearer ' + access_token}

import pandas as pd
from pandas import json_normalize

pg_contents = []
for pg in range(pgs):
    print("Page: {}".format(pg + 1))
    param = {'per_page': act_pp, 'page': pg + 1}
    pg_dataset = requests.get(activites_url, headers=header, params=param).json()
    print(pg_dataset)
    pg_activities = json_normalize(pg_dataset)
    print(pg_activities.head())
    pg_activities = pg_activities[cols]
    print("retrieved {} activities".format(pg_activities.shape[0]))
    #print(pg_activities.head())
    pg_contents.append(pg_activities)
    
my_activities = pd.concat(pg_contents)
print("Total of {} activities retrieved".format(my_activities.shape[0]))

Page: 1
{'message': 'Authorization Error', 'errors': [{'resource': 'AccessToken', 'field': 'activity:read_permission', 'code': 'missing'}]}
               message                                             errors
0  Authorization Error  [{'resource': 'AccessToken', 'field': 'activit...


KeyError: "None of [Index(['id', 'name', 'distance', 'moving_time', 'elapsed_time',\n       'total_elevation_gain', 'type', 'start_date_local', 'map.id',\n       'average_speed', 'max_speed', 'elev_high', 'elev_low', ''],\n      dtype='object')] are in the [columns]"

In [47]:
print(my_activities[""])

           id                name  distance  moving_time  elapsed_time  \
0  6906273468      Afternoon Hike       0.0           88            88   
1  6895088466        Morning Hike    4980.0         4261          7040   
2  6886935853  Afternoon Activity    6536.0         6050         15264   
3  6878705037           Lunch Run    9760.0         3420          3420   
4  6869475041           Lunch run    9766.6         3461          3482   

   total_elevation_gain     type      start_date_local       map.id  \
0                   0.0     Hike  2022-03-30T14:16:20Z  a6906273468   
1                   0.0     Hike  2022-03-27T09:13:53Z  a6895088466   
2                 707.0  Workout  2022-03-26T13:55:13Z  a6886935853   
3                   0.0      Run  2022-03-24T12:47:21Z  a6878705037   
4                  32.3      Run  2022-03-23T12:49:38Z  a6869475041   

   average_speed  max_speed  elev_high  elev_low  
0          0.000      0.000      283.8     282.8  
1          1.169      4.80